Evolución de los Precios de la Electricidad
===

# Instrucciones

En el archivo `precios.zip`  se encuentran los archivos `Precio_Bolsa_Nacional_($kwh)_ano.xls` en formato de Microsoft Excel, los cuales contienen los precios históricos horarios de la electricidad para el mercado eléctrico Colombiano entre los años 1995 y 2017 en COL-PESOS/kWh. A partir de la información suministrada realice las siguientes actividades. 

**1.--** Lea los archivos y cree una tabla única concatenando la información para cada uno de los años. Debe transformar la tabla de tal forma que quede con las columnas `Fecha`, `Hora` y `Precio` (únicamente tres columnas).

**2.--** Compute e imprima el número de registros con datos faltantes.

**3.--** Compute e imprima el número de registros duplicados.

**4.--** Elimine los registros con datos duplicados o datos faltantes, e imprima la cantidad de registros que quedan (registros completos).

**5.--** Compute y grafique el precio primedio diario.

**6.--** Compute y grafique el precio máximo por mes.

**7.--** Compute y grafique el precio mínimo mensual.

**8.--** Haga un gráfico para comparar el precio máximo del mes (para cada mes) y el precio promedio mensual.

**9.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días laborales.

**10.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días sabado.

**11.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días domingo.

**12.--** Imprima una tabla con la fecha y el valor más bajo por año del precio de bolsa.

**13.--** Haga una gráfica en que se muestre el precio promedio diario y el precio promedio mensual.

---

# Solución

In [1]:
# Libraries

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from os import listdir
import glob

%matplotlib inline

**1.--** Lea los archivos y cree una tabla única concatenando la información para cada uno de los años. Debe transformar la tabla de tal forma que quede con las columnas `Fecha`, `Hora` y `Precio` (únicamente tres columnas).

In [88]:
path = 'precios/'
files = listdir(path)
file_names = [f'{path}{file}' for file in files if (file.endswith('xls') or file.endswith('xlsx'))]
file_names.sort()
file_names

['precios/Precio_Bolsa_Nacional_($kwh)_1995.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_1996.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_1997.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_1998.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_1999.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2000.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2001.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2002.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2003.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2004.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2005.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2006.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2007.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2008.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2009.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2010.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2011.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2012.xlsx',
 'precios/Precio_Bolsa_Nacional_($kwh)_2013.xlsx',
 'precios/Precio_Bolsa_Nacional

In [151]:
def read_data_file(path):
    '''
    Reads the data from the xlsx files, deleting nan heads values
    '''
    
    index = 0
    data = pd.read_excel(path, skiprows=index)
    while data.iloc[0,0] != 'Fecha':
        data = pd.read_excel(path, skiprows=index, usecols=list(range(25)))
        index += 1
    data = pd.read_excel(path, skiprows=index, usecols=list(range(25)))
    return data


In [152]:
dfs = []
for file in file_names:
    dfs.append(read_data_file(file))

In [153]:
dfs = []
for file in file_names:
    dfs.append(read_data_file(file))
dfs = pd.concat(dfs)

In [154]:
# Pivots hours (columns) as rows
result=pd.melt(dfs,id_vars=['Fecha'], var_name='Hora', value_name='Precio')
result.tail()

,Fecha,Hora,Precio
201811,2018-05-01,23,64.229226
201812,2018-05-02,23,67.832816
201813,2018-05-03,23,63.710848
201814,2018-05-04,23,64.043453
201815,2018-05-05,23,75.242918


**2.--** Compute e imprima el número de registros con datos faltantes.

In [157]:
result.isnull().sum().sum()

454

---